In [1]:
from IPython.display import display, HTML
display(HTML("""<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{fontsize:12pt;padding:5px;}
table.dataframe{font-size:12px;}))
</style>
"""))

# <span style="color:red">ch09 09 chroma → pinecorn을 활용한 RAG구현(UpstageEmbedding) </span>

# 벡터 DB : Chroma vs Pinecone

- Chroma : 인메모리 vector DB, 로컬메모리 vector DB
- Pinecone : 클라우드 vector DB (Pinecone console에 api key 생성 → .env (PINECONE_API_KEY등록))

# 1. Knowledge Base 구성을 위한 데이터 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200
)
document_list = loader.load_and_split(text_splitter = text_splitter)

In [3]:
len(document_list)

183

In [6]:
# embedding : upstage embedding-query
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
embedding = UpstageEmbeddings(
    model = "solar-embedding-1-large"
#     model = "embedding-query"
)

In [8]:
%%time

# pinecone vector database

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()
# 데이터를 처음 업로드할 때
index_name = "tax-index-upstage"
database = PineconeVectorStore.from_documents(
    documents = document_list,
    embedding = embedding,
    index_name = index_name
)

# 업로드한 벡터DB 가져올 때
# database = PineconeVectorStore(
#     embedding = embedding,  # 질문을 임베딩하여 유사도 검색
#     index_name = index_name
# )

CPU times: total: 11 s
Wall time: 41.4 s


# 2. 제공되는 prompt를 활용하여 답변 생성

In [9]:
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-4.1-nano")

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt":prompt}
)

In [11]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
ai_message = qa_chain.invoke({'query':query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 정확한 금액을 판단하기 어렵지만, 근로소득공제와 공제 한도(최대 2,000만원)를 반영하면 과세표준이 상당히 낮아지고, 이에 따라 세액이 줄어듭니다. 따라서 기본 세율과 공제액을 고려할 때, 소득세는 수백만원 수준일 수 있습니다. 정확한 계산을 위해서는 구체적인 공제 항목과 세율표를 참고해야 합니다.'}